# 📊 Data Workspace Notebook
# Author: David Linger
# Created: October 2025

"""
This notebook is part of the local data workspace environment.
It is designed for exploratory analysis, data cleaning, and model development.

Environment:
- Python 3.14(local install)
- Virtual Environment: data_env
- Kernel: Python (data_env)

Tools & Libraries:
- pandas, numpy, matplotlib, seaborn
- scikit-learn, ipykernel, jupyter

Version Control:
- Managed via Git (local repo)

Notes:
- All paths are local (no cloud sync)
- Virtual environment is excluded from version control via .gitignore
- For reproducibility, install dependencies via requirements.txt

"""


In [45]:
import pandas as pd
import matplotlib as plt
import yfinance as yf
import ta
import time

To start off the dataset we are going to be looking at a small subset of the stock market and will be focussing on ETF's by producing a 5-ticker set for a diversified preliminary dataset that will provide broad exposure to large-cap U.S. equities across different industries. 

1. SPY – SPDR S&P 500 ETF
- Tracks: S&P 500 Index (500 large U.S. companies)
- Focus: Broad U.S. market exposure
- Top Holdings: Apple, Microsoft, Amazon
- Use Case: Core equity benchmark

2. QQQ – Invesco Nasdaq-100 ETF
- Tracks: Nasdaq-100 Index (100 largest non-financial U.S. companies)
- Focus: Tech-heavy growth stocks
- Top Holdings: NVIDIA, Apple, Meta, Google
- Use Case: High-growth, innovation-focused exposure

3. DIA – SPDR Dow Jones Industrial Average ETF
- Tracks: Dow Jones Industrial Average (30 blue-chip companies)
- Focus: Stable, mature U.S. companies
- Top Holdings: UnitedHealth, Goldman Sachs, Boeing
- Use Case: Defensive, value-oriented investing

4. TLT – iShares 20+ Year Treasury Bond ETF
- Tracks: Long-term U.S. Treasury bonds
- Focus: Fixed income, interest rate sensitivity
- Top Holdings: U.S. government bonds
- Use Case: Hedge against equity risk, macro exposure

5. VXUS – Vanguard Total International Stock ETF
- Tracks: Global stocks outside the U.S.
- Focus: International diversification
- Top Holdings: Nestlé, Samsung, Toyota
- Use Case: Exposure to developed and emerging markets

ETF(Exchange-Traded Fund) - is a type of investment fund that can be bought and sold just like a regular stock on a stock exchange. But instead of representing one company, an ETF holds a conjunction of assets, such as stocks, bonds, commodities, or currencies.

Candle Granularity

For the purposes of pensionfund trading were going to be focussing on: 
- end-of-day trading; a model that will be optimized for buy/sell once a day after market close.
- Swing trading; Holding positions for several days to weeks.
- Portfolio rotation:; Rebalance weekly or monthly based on daily signals.

In [26]:
# fetch the tables for the ETFs

ETFS = ["SPY", "QQQ", "DIA", "TLT", "VXUS"]
df_etf = yf.download(ETFS, interval="1d", start="2023-01-01", end="2025-01-01", group_by="ticker")


C:\Users\david.linger\AppData\Local\Temp\ipykernel_23884\3119836584.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_etf = yf.download(ETFS, interval="1d", start="2023-01-01", end="2025-01-01", group_by="ticker")
[*********************100%***********************]  5 of 5 completed


In [27]:
df_etf

Ticker             QQQ                                                \
Price             Open        High         Low       Close    Volume   
Date                                                                   
2023-01-03  264.232022  265.707355  257.819255  260.130615  42335300   
2023-01-04  262.255096  263.051774  258.212670  261.369873  47754900   
2023-01-05  259.697887  259.865075  256.963606  257.278320  45396700   
2023-01-06  259.019191  265.500814  256.058681  264.379547  54659700   
2023-01-09  266.376213  270.762890  265.481204  266.091003  45568700   
...                ...         ...         ...         ...       ...   
2024-12-24  522.854515  528.054837  522.216909  527.965210  17558200   
2024-12-26  526.331417  529.240409  524.328973  527.606567  19090500   
2024-12-27  524.030044  524.468390  515.910697  520.593018  33839600   
2024-12-30  513.569590  517.405074  509.903419  513.669189  34584000   
2024-12-31  514.954366  515.711454  508.339345  509.305695  29117000   

Ticker            TLT                                             ...  \
Price            Open       High        Low      Close    Volume  ...   
Date                                                              ...   
2023-01-03  91.838566  91.964371  90.616447  91.173592  25313200  ...   
2023-01-04  92.755144  92.988786  91.847540  92.422653  21678200  ...   
2023-01-05  91.541992  92.818024  91.461113  92.809036  14771100  ...   
2023-01-06  92.287878  94.624285  92.233963  94.516449  27473700  ...   
2023-01-09  93.824469  95.190362  93.698657  95.019623  21472200  ...   
...               ...        ...        ...        ...       ...  ...   
2024-12-24  84.257642  85.080469  84.199562  85.061111  22377600  ...   
2024-12-26  84.422194  85.148219  84.412512  85.012695  19981800  ...   
2024-12-27  84.683569  84.973975  84.276990  84.315712  27262300  ...   
2024-12-30  85.022387  85.225673  84.867498  84.993347  48519600  ...   
2024-12-31  85.312791  85.457997  84.470606  84.538368  31917300  ...   

Ticker           VXUS                                                    DIA  \
Price            Open       High        Low      Close    Volume        Open   
Date                                                                           
2023-01-03  48.158164  48.480815  47.807855  48.001446   4103100  316.715463   
2023-01-04  48.674405  48.904871  48.351754  48.821903   4198000  316.848877   
2023-01-05  48.360959  48.554550  48.268774  48.397835   3114200  315.334043   
2023-01-06  48.655967  49.568614  48.407063  49.531738   3166800  316.486878   
2023-01-09  50.020321  50.250787  49.799076  49.799076   3391800  321.736563   
...               ...        ...        ...        ...       ...         ...   
2024-12-24  58.599472  58.599472  58.264562  58.520668  11058600  423.891257   
2024-12-26  58.579767  58.737372  58.461561  58.638866   2977800  425.946975   
2024-12-27  58.441867  58.619169  58.313812  58.510818   2758800  425.561508   
2024-12-30  58.136503  58.353211  57.909947  58.116802   4583800  420.511126   
2024-12-31  58.235006  58.372910  57.969047  58.047852   4727900  421.934341   

Ticker                                                   
Price             High         Low       Close   Volume  
Date                                                     
2023-01-03  318.058825  312.837736  315.572144  3910700  
2023-01-04  318.258968  314.600384  316.848877  4137300  
2023-01-05  315.505532  312.599634  313.666718  4350100  
2023-01-06  321.193501  314.324130  320.393188  4800800  
2023-01-09  323.375306  319.097436  319.449951  3864000  
...                ...         ...         ...      ...  
2024-12-24  427.587657  423.288351  427.459167  1431700  
2024-12-26  428.615518  425.897570  428.160858  1867400  
2024-12-27  427.251599  422.448228  424.988281  2429100  
2024-12-30  422.675604  417.595496  420.807617  3858300  
2024-12-31  422.754654  419.097796  420.540771  2442700  

[502 rows x 25 columns]

In [ ]:
# Reset column index
df_flat = df_etf.stack(level=0).reset_index()

# Rename columns for clarity
df_flat.columns = ["Date", "Ticker", "Open", "High", "Low", "Close", "Volume"]

C:\Users\david.linger\AppData\Local\Temp\ipykernel_23884\3616685413.py:2: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_flat = df_etf.stack(level=0).reset_index()


,Date,Ticker,Open,High,Low,Close,Volume
0,2023-01-03,DIA,316.715463,318.058825,312.837736,315.572144,3910700
1,2023-01-03,QQQ,264.232022,265.707355,257.819255,260.130615,42335300
2,2023-01-03,SPY,370.573320,372.559376,364.268061,367.150757,74850700
3,2023-01-03,TLT,91.838566,91.964371,90.616447,91.173592,25313200
4,2023-01-03,VXUS,48.158164,48.480815,47.807855,48.001446,4103100
...,...,...,...,...,...,...,...
2505,2024-12-31,DIA,421.934341,422.754654,419.097796,420.540771,2442700
2506,2024-12-31,QQQ,514.954366,515.711454,508.339345,509.305695,29117000
2507,2024-12-31,SPY,584.785824,585.509524,579.343521,580.989136,57052700
2508,2024-12-31,TLT,85.312791,85.457997,84.470606,84.538368,31917300


In [ ]:
def add_indicators(df):
    # momentum - Detect overbought/oversold conditions
    df = df.copy()
    df["RSI"] = ta.momentum.RSIIndicator(close=["Close"]).rsi()

    # trend - Identify direction and momentum
    macd = ta.trend.MACD(close= df["Close"], window_fast=12, window_slow=26, window_sign=9) # default window params developed by Gerald Appel
    df["MACD"] = macd.macd()
    df["MACD_signal"] = macd.macd.signal()
    df["MACD_diff"] = macd.macd_diff()

    df["sma_20"] = ta.trend.SMAIndicator(close=df["Close"], window=20).sma_indicator()
    df["ema_20"] = ta.trend.EMAIndicator(close=df["Close"], window=20).ema_indicator()

    df["sma_50"] = ta.trend.SMAIndicator(close=df["Close"], window=50).sma_indicator()
    df["ema_50"] = ta.trend.EMAIndicator(close=df["Close"], window=50).ema_indicator()

    # volatility - Gauge price fluctuation and breakout potential

    

    # volume - Confirms price moves with volume strenght

    


Some breakdowns of the Technical Indicators from the book: Technical Analysis from A to Z by Steven B. Achelis

- RSI (Relative strength index): Welles Wilder, New Concepts in Technical Trading
Systems(1978)

"Popular oscilator for comparing the internal strength of a single 'security'. Price following oscilator that ranges between 0 to 100. A popular method of analyzing the RSI is to look for a divergence in which the security is making a new high, but the RSI is failing to surpass its previous high. This divergence is an indication of an impending reversal. When the RSI then turns down and falls below its most recent trough, it is said to have completed a "failure swing." The failure swing is considered a confirmation of the impending reversal."

- MACD

"The MACD is the difference between a 26-day and 12-day exponential moving average. A 9-day exponential moving average, called the "signal" (or "trigger") line is plotted on top of the MACD to show buy/sell opportunities."

There are three main ways to deploy the MACD; Crossover, Overbought/Oversold Conditions and Divergences. 

1.  Crossover: the general rule to follow is that when the MACD falls below the signal line the asset should be sold, conversely when it is above the signal line the asset should be bought. Additionally when the signal rises or falls above/below zero, it is also common practice to buy or sell relative to it's position.

2. Overbought/Oversold: The MACD can also indicate this attributem when the shorter moving average dramatically pulls away from the longer moving average this is a strong indicator that the asset price is overextending and soon will have to move back to more realistic levels. Since these conditions vary from stock to stock we won't be using this, as we're trying to generalize across market regimes and industries for various stocks at once. 

3.  Divergences: an indication that an end of a trend may be near occurs when the MACD diverges from the asset. A "bearish" divergence is when the MACD is going into new lows whilst the prices fail to reach those lows, A "bullish" divergence is when the MACD is making new highs whilst prices fail to rech new highs. both of these are most significant when overbought/oversold levals are high.

- SMA

"A simple, or arithmetic, moving average is calculated by adding the closing price of the security for a number of time periods (e.g., 12 days) and then dividing this total by the number of time periods. The result is the average price of the security over the time period. Simple moving averages give equal weight to each daily price."

The average is calculated by simply aggregating the closing prices of the stock for each candle at closing time, and dividing it by the number of candles within that timeframe. Since the SMA gives equal weight to all prices in the window, it is slower to react to price changes but better in identifying long-term trends. 

- EMA

"An exponential (or exponentially weighted) moving average is calculated by applying a percentage of today's closing price to yesterday's moving average value. Exponential moving averages place more weight on recent prices."
